In [13]:
from keras import applications, optimizers
from keras import backend as K
from keras import models as Models
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model 
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import os
import sys
import time

img_width, img_height = 256, 256
train_data_dir = "train_set"
validation_data_dir = "test_set"
batch_size = 32
epochs = 10

In [14]:
model = applications.VGG16(weights = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_shape = (img_width, img_height, 3))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [15]:
# Freeze the first 5 layers
for layer in model.layers[:5]:
    layer.trainable = False

# Create output
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(10, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

# custom loss function
def customLoss(yTrue,yPred):
    return K.sum(K.log(yTrue) - K.log(yPred))

# compile the model
# model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
model_final.compile(loss = customLoss, optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

print(model_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [16]:
# Save the model according to the conditions  
filepath="my_weight-{epoch:02d}-{loss:.4f}-m1.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=8, verbose=1, mode='auto')

In [17]:
# Initiate the train and test generators with data Augumentation 
train_datagen_rescale = ImageDataGenerator(rescale = 1./255)

train_datagen_aug = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen_rescale = ImageDataGenerator(rescale = 1./255)

test_datagen_aug = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen_aug.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen_aug.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

Found 1450 images belonging to 10 classes.
Found 264 images belonging to 10 classes.


In [18]:
# Train the model
time_start = time.time()

model_final.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator)

time_elapsed = time.time() - time_start

# Record time used
print(time_elapsed)

Epoch 1/10
46/46 [==============================] - 50s 1s/step - loss: -inf - acc: 0.0442 - val_loss: -inf - val_acc: 0.0227
Epoch 2/10
46/46 [==============================] - 44s 951ms/step - loss: -inf - acc: 0.1336 - val_loss: -inf - val_acc: 0.0379
Epoch 3/10
46/46 [==============================] - 44s 962ms/step - loss: -inf - acc: 0.0728 - val_loss: -inf - val_acc: 0.0379
Epoch 4/10
46/46 [==============================] - 44s 951ms/step - loss: -inf - acc: 0.0864 - val_loss: -inf - val_acc: 0.0379
Epoch 5/10
46/46 [==============================] - 44s 957ms/step - loss: -inf - acc: 0.0994 - val_loss: -inf - val_acc: 0.0379
Epoch 6/10
46/46 [==============================] - 47s 1s/step - loss: -inf - acc: 0.0917 - val_loss: -inf - val_acc: 0.0379
Epoch 7/10
46/46 [==============================] - 44s 950ms/step - loss: -inf - acc: 0.0939 - val_loss: -inf - val_acc: 0.0379
Epoch 8/10
46/46 [==============================] - 44s 955ms/step - loss: -inf - acc: 0.1010 - val_los

In [19]:
score = model_final.evaluate_generator(validation_generator)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: -inf
Test accuracy: 0.0378787878788
